# 处理ONEK1K数据

In [2]:
import os
from pyliftover import LiftOver
import modin.pandas as pd
import numpy as np

In [2]:
#根据核对-斯皮尔曼的RHO是A2的
onek1k_eSNP = pd.read_csv('/CIMA/Data/public_eQTL/onek1k/esnp_table.tsv',sep='\t')
# onek1k_eSNP
celltype_mapping = {
    'cd4nc': 'CD4_T', 'cd4sox4': 'CD4_T', 'cd4et': 'CD4_T',
    'cd8s100b': 'CD8_T', 'cd8et': 'CD8_T', 'cd8nc': 'CD8_T',
    'bmem': 'B', 'bin': 'B', 'plasma': 'B',
    'nk': 'NK', 'nkr': 'NK',
    'monoc': 'Myeloid', 'mononc': 'Myeloid', 'dc': 'Myeloid'
}

2025-02-19 12:15:08,011	INFO worker.py:1749 -- Started a local Ray instance.


In [3]:
onek1k_eSNP['celltype_map'] = onek1k_eSNP['CELL_ID'].map(celltype_mapping)
onek1k_eSNP = onek1k_eSNP[onek1k_eSNP.loc[:,'ROUND'] == 1]

In [4]:
lo = LiftOver('hg19', 'hg38')
def convert_coordinates(row):
    hg_38_liftover = lo.convert_coordinate(row['CHR'], row['POS'])
    
    if len(hg_38_liftover) == 0:
        row['POS'] = np.nan
    elif len(hg_38_liftover) >= 1:
        row['CHR'] = hg_38_liftover[0][0]
        row['POS'] = hg_38_liftover[0][1]
    
    return row

In [5]:
onek1k_eSNP.loc[:,'CHR'] = onek1k_eSNP['CHR'].astype('str')
onek1k_eSNP.loc[:,'CHR'] = 'chr'+onek1k_eSNP['CHR'] 
onek1k_eSNP['CELLTYPE_GENE'] = onek1k_eSNP['celltype_map']+'_'+onek1k_eSNP['GENE']

In [6]:
onek1k_eSNP_hg38 = onek1k_eSNP.apply(convert_coordinates, axis=1)
onek1k_eSNP_hg38.dropna(subset=['POS'],inplace=True)
onek1k_eSNP_hg38.loc[:,'POS'] = onek1k_eSNP_hg38['POS'].astype('int')
onek1k_eSNP_hg38.loc[:,'POS'] = onek1k_eSNP_hg38['POS'].astype('str')

In [7]:
min_p_value_rows = onek1k_eSNP_hg38.loc[onek1k_eSNP_hg38.groupby('CELLTYPE_GENE')['P_VALUE'].idxmin()].reset_index(drop = True)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [8]:
min_p_value_rows.to_csv('/CIMA/Data/public_eQTL/20250218_file_38_for_comparasion/20250218_onek1k_leap_snp.csv')

# 处理ImmueNexut数据

In [1]:
import os
from pyliftover import LiftOver
import pandas as pd
import numpy as np

In [20]:
folder_path = '/CIMA/Data/public_eQTL/ImmuNexUT/conditional/'
# 列出所有 .txt 文件
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

In [21]:
celltype_list = list(pd.Series(txt_files).str.replace('_conditional_eQTL_FDR0.05.txt',''))

In [22]:
celltype_mapping = {
    'Naive_CD4': 'CD4_T', 'Mem_CD4': 'CD4_T', 'Th1':'CD4_T','Th2':'CD4_T','Tfh': 'CD4_T', 'Th17': 'CD4_T','Fr_III_T': 'CD4_T', 'Fr_II_eTreg': 'CD4_T', 'Fr_I_nTreg': 'CD4_T',#CD4_T细胞相关
    'CM_CD8': 'CD8_T', 'EM_CD8': 'CD8_T', 'Naive_CD8': 'CD8_T', 'Mem_CD8': 'CD8_T', 'TEMRA_CD8': 'CD8_T',  # CD8+ T细胞相关
    'Naive_B': 'B', 'DN_B': 'B', 'SM_B': 'B', 'USM_B': 'B','Plasmablast': 'B',  # B细胞相关
    'NK': 'NK',
    'CD16p_Mono': 'Myeloid', 'CL_Mono': 'Myeloid', 'Int_Mono': 'Myeloid', 'NC_Mono': 'Myeloid', 'Neu': 'Myeloid', 'mDC': 'Myeloid', 'pDC': 'Myeloid','LDG':'Myeloid' # 单核细胞相关
      # NK细胞
}

In [23]:
df_combined = pd.DataFrame()
#只选rank0而且是最强那一个
for i in range(0,28,1):
    snp_list = pd.read_csv(f'/CIMA/Data/public_eQTL/ImmuNexUT/conditional/{txt_files[i]}',sep='\t')

    snp_list = snp_list[snp_list['Rank_of_association'] == 0].reset_index(drop = True)

    snp_list =snp_list.iloc[snp_list.groupby('Gene_name')['Forward_nominal_P'].idxmin().values,:]

    snp_list=snp_list.reset_index(drop = True)

    snp_list['CELL_TYPE'] = celltype_list[i]

    snp_list['pair'] = snp_list['Gene_name']+'_'+snp_list['Variant_CHR']+'_'+snp_list['Variant_position_start'].astype(str)

    snp_list_full = pd.read_csv(f'/CIMA/Data/public_eQTL/ImmuNexUT/norminal/{celltype_list[i]}_nominal.txt',sep='\t')
    
    snp_list_full['pair'] = snp_list_full['Gene_name']+'_'+snp_list_full['Variant_CHR']+'_'+snp_list_full['Variant_position_start'].astype(str)

    snp_list_full = snp_list_full[['REF','ALT','nominal_P_value','slope(ALT)','pair']]

    snp_list = pd.merge(snp_list, snp_list_full, on='pair', how='inner')  # 默认是 'inner' 合并

    snp_list = snp_list.reset_index(drop=True)

    df_combined = pd.concat([df_combined, snp_list], axis=0, ignore_index=True)

In [25]:
df_combined['celltype_map'] = df_combined['CELL_TYPE'].map(celltype_mapping)
df_combined['CELLTYPE_GENE'] = df_combined['celltype_map']+'_'+df_combined['Gene_name']

In [28]:
df_combined.to_csv('/CIMA/Data/public_eQTL/20250218_file_38_for_comparasion/20250219_immuenexut_all_snp.csv')

In [26]:
min_p_value_rows = df_combined.loc[df_combined.groupby('CELLTYPE_GENE')['Forward_nominal_P'].idxmin()].reset_index(drop = True)

In [9]:
min_p_value_rows.to_csv('/CIMA/Data/public_eQTL/20250218_file_38_for_comparasion/20250219_immuenexut_all_lead_snp.csv')